In [ ]:
# import pandas as pd
import numpy as np
import missingno as msno
import janitor
import dataprep as dp
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
data=pd.read_csv(r"C:\Users\uditb\Downloads\smartphones\mobiles_raw.csv")
data.head(5)

In [ ]:
cols_to_drop=[
    "parent_url","description","box_content", "model_num","sims","sims_hybrid","otg","sound_enhance", "disp_res_type",
    "disp_type","disp_cols", "cpu_type", "cpu_clock_2", "mem_slot",
    "cam_prim_feat","cam_opt_zoom", "cam_secd_feat", "cam_flash","zoom","cam_fps",
    "musb","infrared","sim_size","user_intf","disp_ppi","sensors","tones","gps_type","radio", 
    "bat_type","dim_width","dim_height","dim_depth","warranty","model_colour","cpu_clock_1",
    "storage_exp"
    ]
len(cols_to_drop)

In [ ]:
data.drop(columns=cols_to_drop,inplace=True)

In [ ]:
data.head()

In [ ]:
## since there are alot of NA values in discount and marked_price_inr we will rely on price_inr
# for our analysis
print(data['discount'].isna().sum())
print(data['price_inr'].isna().sum())
print(data['marked_price_inr'].isna().sum())

In [ ]:
data.info()

In [ ]:
# data.memory_usage(deep=True)

### price_inr and marked_price_inr

In [ ]:
## removing Rupee Symbol and , from price columns
data['price_inr']=data['price_inr'].str.replace("₹","").str.replace(",","")
data['marked_price_inr']=data['marked_price_inr'].str.replace("₹","").str.replace(",","")

## Creating an Indepth Report of Missing Data

In [ ]:
from dataprep.eda.missing import plot_missing
report=plot_missing(data)
report

In [ ]:
report.save("data_missing_report_no_cleaning.html")

In [ ]:
## we will drop duplicates. Mostly we see duplicates as different colors of the same phone
print(data[data.duplicated()].shape)
data[data.duplicated()].head()

In [ ]:
data.drop_duplicates(keep="first",inplace=True)

In [ ]:
# we had 6983 rows and are left with 6924 rows after removing duplicates
data.shape

In [ ]:
# checking rows where all values are missing
data[data.isna().all(axis=1)]

In [ ]:
## here all three columns price_inr, marked_price_inr and discount values are NA we will drop them
price_missing_index=data.query("price_inr.isna()&marked_price_inr.isna()&discount.isna()").index
data=data.drop(index=price_missing_index)
data.head()

In [ ]:
## changing price to an integer
data['price_inr']=data['price_inr'].astype("int")

# Data Cleaning

In [ ]:
# we can see in name column contains other characteristics like color, storage, RAM. We will extract the name only and store it
data['name']=data['name'].str.lower().str.split("(").str.get(0)
data.head()

In [ ]:
data.drop(columns=['marked_price_inr','discount'],inplace=True)
data.head()

### stars

In [ ]:
data.stars.isna().sum()

In [ ]:
data['stars']=data['stars'].astype("float")

In [ ]:
data.rating_count.isna().sum()

In [ ]:
data.review_count.isna().sum()

In [ ]:
data.drop(columns=['review_count','model_name'],axis=1,inplace=True)

### rating_count

In [ ]:
data['rating_count']

In [ ]:
data['rating_count'].isna().sum()

In [ ]:
# cleaning this columns
data['rating_count']=(
    data['rating_count']
    .str.lower()
    .str.split(" ")
    .str.get(0)
    .str.replace(",",'')
)

In [ ]:
data['rating_count']

In [ ]:
# from tabulate import tabulate
# markdown_table = tabulate(data['rating_count'],tablefmt='pipe',)
# markdown_table

In [ ]:
data['rating_count']=data['rating_count'].astype('float')

### quick_charge

In [ ]:
#quick charge has alot of missing values
data['quick_charge'].value_counts(dropna=False)

In [ ]:
## Random Text values in quick charge column
vals_to_fix=[4218,4219,4222,4223]
temp_df=data.loc[vals_to_fix,:]
temp_df

In [ ]:
## if the column contained quick charge for these I replace them as Yes
temp_df.loc[temp_df['quick_charge'].str.contains("Quick Charging", na=False,case=False), 'quick_charge']="Yes"

In [ ]:
## here I replaced the values in the original data with the values assigned above
data.loc[temp_df.index,'quick_charge']=temp_df['quick_charge'].values

In [ ]:
data ['quick_charge'] = (
    data ['quick_charge'].str.lower().map (
        { 'yes' :1,
        'no' :0 }
    )
)

In [ ]:
data['quick_charge'].value_counts(dropna=False)

### disp_size

In [ ]:
data['disp_size']

In [ ]:
## this extracts screen size inside parenthesis and removes the inches word to give us a float number value we can use
data['disp_size']=data['disp_size'].str.extract(r'\(([\d.]+) inch\)')[0]

In [ ]:
data['disp_size'].isna().sum()

In [ ]:
data['disp_size']=data['disp_size'].astype("float")

In [ ]:
data['disp_size']

### disp_res

In [ ]:
data['disp_res'].sample(10)

In [ ]:
# first step is to remove text Pixels
disp_res=(
    data['disp_res']
    .str.lower()
    .str.replace(r'Pixels?', '', regex=True,case=False)
)
disp_res

In [ ]:
# this first uses regex to find and replace special characters at the end and removes them. 
data['disp_res']=(
    disp_res
    .str.replace(r'\D+', ' x ', regex=True)
    .str.replace(r'\s*x\s*$', '', regex=True)
)

In [ ]:
disp_res.value_counts()

In [ ]:
## this function will help us get disp_height and disp_width columns
def disp_res_format(res):
    if pd.isna(res):  # Handle NaN values
        return None
    res = str(res)  # Convert to string if it's not
    nums = [int(x) for x in res.split(' x ') if x.isdigit()]
    if len(nums) == 2:  # Ensure exactly two numbers
        nums.sort(reverse=True)  # Ensure width is first and then height
        return f"{nums[0]} x {nums[1]}"
    return res  # Return original value if needed

data['disp_res']=data['disp_res'].apply(disp_res_format)

In [ ]:
data['disp_res']

In [ ]:
## here we can see disp_res has 2 different resolutions that I will manually fix
data.loc[493]

In [ ]:
data.loc[493,'disp_res']='800 x 480'

In [ ]:
## creating 2 different columns for display_width and display_height
data['display_width']=(
    data['disp_res']
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
    .str.split("x").str.get(0)
)

data['display_height']=(
    data['disp_res']
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
    .str.split("x")
    .str.get(1)
)

In [ ]:
data.drop(columns='disp_res',inplace=True)

In [ ]:
print(data['display_width'].isna().sum())
print(data['display_height'].isna().sum())

In [ ]:
data[['display_width','display_height']].head(5)

### gpu

In [ ]:
data['gpu'].value_counts()

In [ ]:
## since more than 50% of the column contains null values I will drop this column
data['gpu'].isna().sum() / data.shape[0]

In [ ]:
data.drop(columns='gpu',inplace=True)

### os_name

In [ ]:
data['os_name'].unique()

In [ ]:
data.os_name.value_counts(dropna=False)

In [ ]:
## os_name values that need to be fixed
values_to_inspect=[
    1294, 1299, 1302, 1303, 1304, 1313, 1339, 1340, 1342, 1347, 1348,
    1354, 1367, 1374, 1380, 1392, 1394, 1400, 1404, 1407, 1442, 1455,
    1456, 1460, 1464, 1465, 1467, 1472, 1494, 1496, 1500, 1506, 1508,
    1510, 1511, 1514, 1528, 1533, 1542, 1543, 1548, 1550, 1553, 1558,
    1559, 1560, 1562, 1570, 1574, 1578, 1581, 1588, 1590, 1591, 1594,
    1596, 1604, 1606, 1613, 1665, 1677, 1851, 1856, 1858, 1889, 1890,
    1909, 1921, 1923, 1938, 1951, 1959, 1964, 1971, 1990, 1991, 1992,
    2190, 2222, 2309, 2347, 2350, 2354, 2359, 2361, 2363, 2561, 2566,
    2570, 2574, 2575, 2600, 2601, 2638, 2652, 2653, 2658, 2713, 2714,
    2724, 2725, 2860, 2874, 2880, 2884, 2932, 2933, 2934, 2935, 2936,
    2937, 2945, 2946, 2949, 2955, 2967, 2974, 2975, 2984, 2985, 2986,
    2987, 2989, 2990, 2991, 2995, 3007, 3011, 3012, 3015, 3023, 3027,
    3028, 3059, 3060, 3072, 3073, 3084, 3088, 3090, 3093, 3095, 3101,
    3120, 3130, 3140, 3171, 3173, 3179, 3190, 3200, 3210, 3212, 3213,
    3224, 3229, 3236, 3238, 3241, 3243, 3247, 4707, 5378, 5386, 5394,
    6208, 6259, 6286, 6291, 6324, 6335, 6411, 6414, 6416
    ]

In [ ]:
len(values_to_inspect)

In [ ]:
temp_df=data.loc[values_to_inspect,:]
temp_df['os_name']

In [ ]:
# replacing the values above as other
data.loc[temp_df.index,'os_name']='other'

In [ ]:
## replacing all rows that contain android os with lowercase android
data.loc[data['os_name'].str.lower().str.strip().str.contains('android', na=False), 'os_name'] = 'android'


In [ ]:
## similarly all rows that contain ios os with lowercase ios
data.loc[data['os_name'].str.lower().str.strip().str.contains('ios', na=False), 'os_name'] = 'ios'


In [ ]:
temp_df=data[data['os_name'].str.lower().str.contains("windows",na=False)]

In [ ]:
data.loc[temp_df.index,'os_name']='windows'

In [ ]:
temp_df=data[~data['os_name'].isin(['android','ios','windows'])]
temp_df.index

In [ ]:
data.loc[temp_df.index,'os_name']='other'

In [ ]:
data['os_name'].sample(10)

In [ ]:
data['os_name'].value_counts()

In [ ]:
data.head()

### cpu_core

In [ ]:
data.head()

In [ ]:
data['cpu_core'].unique()

In [ ]:
data['cpu_core'].unique()

In [ ]:
data['cpu_core']=data['cpu_core'].str.lower().map(
    {
        'single core':1,
        'octa core':8,
        'dual core':2,
        'quad core':4,
        'hexa core':6,
        'deca core':10
    }
)

In [ ]:
data['cpu_core']=data['cpu_core'].astype("float")

In [ ]:
data['cpu_core'].value_counts(dropna=False)

### ram

In [ ]:
data['ram'].sample(10)

In [ ]:
# in the rows below all ram capacity is in MB and I will convert it to GB
temp_df=data[data['ram'].str.contains('MB',na=False,case=False)]
temp_df.head()

In [ ]:
ram_vals_gb=temp_df['ram'].str.lower().str.replace('MB','',case=False).astype('float')
ram_vals_gb=ram_vals_gb.div(1000)

In [ ]:
temp_df['ram']=ram_vals_gb.values

In [ ]:
data.loc[temp_df.index,'ram']=temp_df['ram'].values

In [ ]:
data['ram']=data['ram'].str.lower().str.replace('GB','',case=False)

In [ ]:
data['ram'].value_counts(dropna=False)

In [ ]:
data['ram']=data['ram'].astype("float")

### storage_int

In [ ]:
data['storage_int'].sample(10)

In [ ]:
# these smartphones have internal storage in MB and I will convert them to GB
temp_df=(
    data[
        data['storage_int']
        .str.contains('MB',case=False,na=False)
        ]
)
temp_df.head()

In [ ]:
temp_df.loc[[6332],'storage_int']

In [ ]:
import numpy as np
temp_df.loc[[6332],'storage_int']=np.nan

In [ ]:
temp_df['storage_int']=(
    temp_df['storage_int']
    .str.replace('MB','',case=False)
    .str.strip().astype('float').div(1000)
    .astype('str')
)

In [ ]:
data.loc[temp_df.index,'storage_int']=temp_df['storage_int'].values

In [ ]:
tb_storage=(
    data[
        data['storage_int']
        .str.contains('TB',case=False,na=False)
        ]
)
tb_storage

In [ ]:
tb_storage['storage_int']=(
    tb_storage['storage_int']
    .str.replace('TB','',case=False)
    .astype('int')
    .multiply(1000)
    .astype('str')
)

In [ ]:
tb_storage

In [ ]:
data.loc[tb_storage.index,'storage_int']=tb_storage['storage_int'].values

In [ ]:
tb_storage['storage_int'].values

In [ ]:
kb_storage_index=(
    data[
        data['storage_int']
        .str.contains('KB',case=False,na=False)]
        .index
)

In [ ]:
data.loc[kb_storage_index,'storage_int']=np.nan

In [ ]:
temp_df=data[data['storage_int'].isin(['32+3 GB', '16 GB/8 GB','64GB GB'])]
temp_df

In [ ]:
temp_df.loc[1794,'storage_int']='16 GB'

In [ ]:
temp_df.loc[2508,'storage_int']='64 GB'

In [ ]:
temp_df.loc[[6483,6503],'storage_int']='32 GB'

In [ ]:
data.loc[temp_df.index,'storage_int']=temp_df['storage_int'].values

In [ ]:
data['internal_storage_gb']=(
    data['storage_int']
    .str.replace('GB','',case=False)
    .astype('float')
)

In [ ]:
data['internal_storage_gb']=data['internal_storage_gb'].astype("float")

In [ ]:
vals=data['internal_storage_gb'].value_counts(dropna=False,ascending=False)
vals

In [ ]:
data.drop(columns=['storage_int'],inplace=True)

### high_refresh_rate

In [ ]:
data['disp_feat']

In [ ]:
## adding a new column high refrsh rate from disp_feat columns that contains the word high refresh rate or 120 hz or pro motion as used 
# by apple
data['high_refresh_rate']=(
    data['disp_feat']
    .str.contains(r'\b(high refresh rate|120\s*hz|pro\s*motion)\b', case=False, na=False)
    .astype("int")
    )


In [ ]:
data['high_refresh_rate'].value_counts(dropna=False,ascending=False)

In [ ]:
data.drop(columns=['disp_feat'],axis=1,inplace=True)

In [ ]:
data.head()

### cam_prim

In [ ]:
data['cam_prim'].sample(10)

In [ ]:
rear_cameras=(
    data['cam_prim']
    .str.lower()
    .str.replace(r'[^0-9+mp]','',regex=True)
    .str.split("+")
)

In [ ]:
rear_cameras

In [ ]:
rear_cameras=(
    data['cam_prim']
    .str.strip() # removes extra spaces
    .str.replace(r'[^0-9+MP]', '', regex=True) # removes all non MP characters as needed
    .str.findall(r'\d+MP') # only finds items containing MPs
)
rear_cameras

In [ ]:
data["rear_primary"]=(
    rear_cameras
    .str.get(0)
    .str.lower()
    .str.strip()
    .str.replace('mp','')
)

In [ ]:
data['rear_primary']=data['rear_primary'].astype('float')

In [ ]:
data['rear_primary'].value_counts(dropna=False)

In [ ]:
data.drop(columns=['cam_prim'],inplace=True)

In [ ]:
data.head()

### cam_secd

In [ ]:
data['cam_secd']

In [ ]:
## this regex pattern will exteract one or more digits followed by MP
data['front_camera_res_mp']=data['cam_secd'].str.strip().str.extract(r'(\d+MP)')

In [ ]:
## we remo the 'MP' str from front camera resolution
data['front_camera_res_mp']=data['front_camera_res_mp'].str.replace('MP','')
data['front_camera_res_mp']

In [ ]:
data['front_camera_res_mp']=data['front_camera_res_mp'].astype('float')
data['front_camera_res_mp']

In [ ]:
data['front_camera_res_mp'].value_counts(dropna=False,ascending=False)

In [ ]:
## since most smartphones have HD video recording I will drop these columns
data.drop(columns=['cam_secd','recd_hd','rec_fhd'],inplace=True)

In [ ]:
data.head()

### rec_vid_res

In [ ]:
data['rec_vid_res'].sample(10)

In [ ]:
data['rec_vid_res'].isna().sum()

In [ ]:
search_4k=['3840x2160 pixels','3840x2160','4k','2160p','uhd','Ultra HD']
fhd_search=['1920x1080','1080p','fhd','1080','1920 x 1080 Pixels','1080 x 1920']
rec_8k=['8K','7680 x 4320 pixel']

# creating a search patter for 4k and 8k video recording to see if it can impact smartphone pricing
search_4k_pattern="|".join(search_4k)
search_8k_pattern='|'.join(rec_8k)
fhd_search_pattern = '|'.join(fhd_search)

In [ ]:
data['vid_rec_res_max']=(
    np.where(
        data['rec_vid_res'].str.contains(search_8k_pattern, case=False, na=False), '8K',
        np.where(
        data['rec_vid_res'].str.contains(search_4k_pattern, case=False, na=False), '4K',
        np.where(
        data['rec_vid_res'].str.contains(fhd_search_pattern, case=False, na=False), 'FHD', np.nan
        )
        )
        )
)

In [ ]:
data['vid_rec_res_max'].value_counts(dropna=False)

In [ ]:
data.drop(columns=['rec_vid_res'],inplace=True)

### networks

In [ ]:
data['networks']

In [ ]:
## if a smartphone is 5G capable then we enter 1 else 0
data['5g_capable']=(
    data['networks']
    .str.contains('5G',na=False,case=False)
    .map(
    {
        True: 1,
        False:0
    }
    )
).astype('int')

In [ ]:
data.drop(columns=['networks','bluetooth','wifi'],inplace=True)

In [ ]:
data.head()

In [ ]:
data['5g_capable'].value_counts(dropna=False)

In [ ]:
data['5g_capable']=data['5g_capable'].astype("int")

### nfc

In [ ]:
## some random values present in column besides yes, no or missing that we need to clean
data['nfc'].value_counts()

In [ ]:
data.nfc.unique()

In [ ]:
# creating a list of values in NFC column thats not na, yes or no so that I can correct them if needed
unique_values = data['nfc'].dropna().unique()  # Get unique values excluding na
filtered_values = [val for val in unique_values if val not in ['Yes', 'No']]
temp_df=data[data['nfc'].isin(filtered_values)]
temp_df

In [ ]:
## if nfc word is present I label them as Ye
temp_df.loc[temp_df['nfc'].str.contains('NFC',na=False,case=False),'nfc']='Yes'

In [ ]:
# replacing values entered above in data from temp_df
data.loc[temp_df.index,'nfc']=temp_df['nfc'].values

In [ ]:
## Verifying to see if the above commands have worked
data['nfc'].unique()

In [ ]:
data['nfc']=data['nfc'].map(
    {
        'No':0,
        'Yes':1
    }
).astype('float')

In [ ]:
data['nfc'].value_counts(dropna=False)

### headphone jack

In [ ]:
data['jack'].sample(10)

In [ ]:
data['jack'].unique()

In [ ]:
jack_vals=data['jack'].dropna().unique()
vals_filtered=[val for val in jack_vals if val not in [
    '3.5 mm', '3.5 MM','Yes','yes','YES','3.5mm Jack',
    '3.5 MM Jack', '3.5 mm jack','3.5 mm Audio jack','3.55 MM','3.55MM','Yes, 3.5mm', '3.5mm',
    '3.5','3.5MM','3.5mm Stereo Headset Connector','3.5 mm Stereo' '3.5MM STEREO','3.5mm jack',
    '3.5MM STEREO', '3.5 mm Stereo', 'no','No'
    ]
    ]
temp_df=data[data['jack'].isin(vals_filtered)]
temp_df.head()

In [ ]:
## these phones dont have a separate headphone jack so I put no
temp_df.loc[:,'jack']='no'

In [ ]:
temp_df.head()

In [ ]:
## replacing the same rows in dataframe 
data.loc[temp_df.index,'jack']=temp_df['jack'].values

In [ ]:
# if value is not na or in temp_df.index then all rows are labelled as yes for headphone jack
jack_missing_index=set(data[data['jack'].isna()].index)
jack_present_index=list(set(data.index).difference(set(temp_df.index)) - jack_missing_index)
data.loc[jack_present_index,'jack']='yes'

In [ ]:
data['jack'].value_counts(dropna=False)

In [ ]:
## verifying na vales +plus jack present values + values we manually corrected equal data len
len(jack_missing_index) + len(jack_present_index) + len(temp_df)

In [ ]:
data['jack']=data['jack'].map(
    {
        'yes':1,
        'no':0
    }
).astype('float')

In [ ]:
data['jack'].value_counts(dropna=False)

### bat_cap

In [ ]:
data['bat_cap'].sample(10)

In [ ]:
data.head()

In [ ]:
data['bat_cap'].value_counts()

In [ ]:
## extacting number value for battery capacity
data['battery_capacity_mah']=(
    data['bat_cap']
    .str.lower()
    .str.strip()
    .str.split(" ")
    .str.get(0)
)
data.drop(columns=['bat_cap'],axis=1,inplace=True)

In [ ]:
data['battery_capacity_mah']=data['battery_capacity_mah'].astype('float')

In [ ]:
data['battery_capacity_mah'].value_counts(dropna=False)

In [ ]:
data['battery_capacity_mah'].value_counts(dropna=False).sort_values(ascending=False)

### dim_weight

In [ ]:
data['dim_weight'].sample(10)

In [ ]:
## this is a list of values I found that need to be fixed
vals=[ 521,524,641,657,658,1040,1366,1977,2269,3023,3028,3031]

In [ ]:
data.loc[vals,'dim_weight']

In [ ]:
data.loc[[2281,2340],'dim_weight']=np.nan

In [ ]:
data.loc[[2281,2340],'dim_weight']=np.nan

In [ ]:
## these rows contained weight as 0.45 kg so I replaced them with 450 g for consistency
data.loc[[4928, 4934, 4935, 4937],'dim_weight']='450 g'

In [ ]:
temp_df=data.loc[vals,:]
temp_df

In [ ]:
temp_df.loc[[1366],'dim_weight']=156

In [ ]:
temp_df.loc[[1040],'dim_weight']

In [ ]:
weights=temp_df.loc[[521,524,641,657,658],'dim_weight']
weights

In [ ]:
def extract_weight(value):
    parts = value.split(",")  # Split by commas
    for part in parts:
        if "Weight" in part:  # Find the section with 'Weight'
            return part.split(":")[-1].strip().split(" (")[0]  # Extract "160 g" and remove extra text
    return None  # Return None if no weight found

weights_updated=weights.apply(extract_weight)
weights_updated

In [ ]:
temp_df.loc[[521,524,641,657,658],'dim_weight']=weights_updated.values

In [ ]:
temp_df.loc[[1977,2269,3023,3028,3031],'dim_weight']=np.nan

In [ ]:
data.loc[temp_df.index,'dim_weight']=temp_df['dim_weight'].values

In [ ]:
data.loc[[1040],'dim_weight']=192

In [ ]:
data.loc[[4734],'dim_weight']='174 g'

In [ ]:
# verifying that we correctly replaced the dim_weight with temp_df values above
data.loc[temp_df.index,'dim_weight']

In [ ]:
data.loc[[6732, 6904],'dim_weight']='185.5 g'

In [ ]:
data.loc[[4794],'dim_weight']='511 g'

In [ ]:
data['dim_weight']=(
    data['dim_weight']
    .str.strip()
    .str.lower()
    .str.replace('g','')
)

In [ ]:
data['dim_weight']=data['dim_weight'].astype("float")

In [ ]:
data['dim_weight'].value_counts(dropna=False)

### brand

In [ ]:
data['brand']=data['brand'].str.lower()

In [ ]:
## if we drop rows with na values our dataset only have 6 rows left out of 6911
print(data.dropna().shape)
print(data.shape)

## Data Report

In [ ]:
plot_missing(data)

In [ ]:
# Data Preprocessing
data.info()

In [ ]:
data.to_csv('smartphone_data_cleaned.csv',index=False)